# Library importation

In [1]:
import numpy as np
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import re
import ast 
import json

# Recupération de la liste des brevets marqués

In [2]:
list_patents = pd.read_csv('./Data/patents_number.csv',index_col=None)

In [3]:
L = list(list_patents.patent_number.values)

In [4]:
df = pd.DataFrame(columns=['patent_number', 'patent_date', 'patent_processing_time', 'ipc_section','ipc_class','assignee_id',])

# Récuperation des informations des brevets marqués

In [5]:
i=0
while i < (len(L)-700):    
    token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
    token2 = '}&f=["patent_number","patent_date","patent_processing_time","assignee_id","ipc_section","ipc_class","wipo_field_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
    url = token1 + str(L[i:i+700]) + token2
    r = requests.get(url)
    html=r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["patents"]
    df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
    i+=700
    print("{}% done".format(i/len(L)*100))
token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
token2 = '}&f=["patent_number","patent_date","patent_processing_time","assignee_id","ipc_section","ipc_class","wipo_field_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
url = token1 + str(L[i:]) + token2
r = requests.get(url)
html=r.text
html = html.replace("null","None")
Dict = ast.literal_eval(html)["patents"]
df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
df = df.reset_index(drop=True)

2.8957928267074835% done
5.791585653414967% done
8.687378480122451% done
11.583171306829934% done
14.478964133537417% done
17.374756960244902% done
20.270549786952387% done
23.166342613659868% done
26.06213544036735% done
28.957928267074834% done
31.853721093782315% done
34.749513920489804% done
37.64530674719729% done
40.54109957390477% done
43.43689240061225% done
46.332685227319736% done
49.22847805402722% done
52.1242708807347% done
55.02006370744219% done
57.91585653414967% done
60.81164936085715% done
63.70744218756463% done
66.60323501427212% done
69.49902784097961% done
72.39482066768709% done
75.29061349439458% done
78.18640632110206% done
81.08219914780955% done
83.97799197451702% done
86.8737848012245% done
89.769577627932% done
92.66537045463947% done
95.56116328134696% done
98.45695610805444% done


In [6]:
df["marked"]=[1]*len(df)

In [7]:
assignee_id=[]

for index,j in enumerate(df.assignees.values):
    assignees=[]
    for element in j:
        assignees.append(element["assignee_id"])
    if len(assignees)>1:
        assignee_id.append(assignees)
    else:
        assignee_id.append(assignees[0])
df["assignee_id"] = assignee_id
df = df.drop(['assignees'],axis=1)

In [8]:
df

,patent_number,patent_date,patent_processing_time,ipc_section,ipc_class,assignee_id,IPCs,wipos,marked
0,5387603,1995-02-07,433,NaN,NaN,5330279e-a21e-438c-80d1-7e6ae793c092,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1
1,5771295,1998-06-23,552,NaN,NaN,2986e46e-7937-477f-9bc5-1f5eec05183f,"[{'ipc_section': 'H', 'ipc_class': '04'}]",[{'wipo_field_id': '2'}],1
2,5585587,1996-12-17,832,NaN,NaN,f7152515-d420-42d9-a7b6-3a2796bc5d10,"[{'ipc_section': 'G', 'ipc_class': '10'}]","[{'wipo_field_id': '34'}, {'wipo_field_id': '2...",1
3,5321490,1994-06-14,568,NaN,NaN,ec72750e-f789-4749-91f7-6292f7c6fda7,"[{'ipc_section': 'G', 'ipc_class': '01'}]","[{'wipo_field_id': '10'}, {'wipo_field_id': '1...",1
4,5750497,1998-05-12,1161,NaN,NaN,9a3afa40-66e5-4921-b831-64287f2052e0,"[{'ipc_section': 'C', 'ipc_class': '07'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1
...,...,...,...,...,...,...,...,...,...
23905,10299691,2019-05-28,225,NaN,NaN,1b2d67d2-f1f9-4c29-8cd1-a5e75ba27be7,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...",[{'wipo_field_id': '13'}],1
23906,10292890,2019-05-21,966,NaN,NaN,f0997ba8-1ec2-4c0c-aaea-d80066a49317,"[{'ipc_section': 'A', 'ipc_class': '61'}]",[{'wipo_field_id': '13'}],1
23907,10299968,2019-05-28,343,NaN,NaN,e5fc32c3-a913-455a-94ce-9932d30eefff,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...","[{'wipo_field_id': '13'}, {'wipo_field_id': '6'}]",1
23908,10149616,2018-12-11,2133,NaN,NaN,4eb39673-7f59-4c45-bf6c-2275dccf9048,"[{'ipc_section': 'A', 'ipc_class': '61'}]",[{'wipo_field_id': '13'}],1


# Constructing the list of assignees

In [9]:
assignees = []
for el in df.assignee_id:
    if type(el)==list:
        for i in el:
            if i not in assignees:
                assignees.append(i)
    elif el!=None and el not in assignees:
        assignees.append(el)

In [16]:
L_assignees = []
k=0
while k < len(assignees)-100:
    L_assignees.append(json.dumps(assignees[k:k+100]))
    k+=100
L_assignees.append(json.dumps(assignees[k:]))

# Récuperation des brevets non marqués de ces entreprises 

In [44]:
df_list_assignee = pd.DataFrame()
for i, el in enumerate(L_assignees):
    token1 = 'https://api.patentsview.org/assignees/query?q={"assignee_id":'
    token2 = '}&f=["patent_number","assignee_id"]&o={"page":'
    token3 = ',"per_page":10000}'
    url = token1 + el + token2 + str(1) + token3
    r = requests.get(url)
    html=r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["assignees"]
    df_list_assignee = pd.concat([df_list_assignee,pd.read_json(json.dumps(Dict),orient='records')])
    print(i*100)

0
100
200
300


KeyboardInterrupt: 

In [ ]:
added_elements=[]
for i, el in enumerate(L_assignees):
    page = 1
    x = True
    token1 = 'https://api.patentsview.org/patents/query?q={"assignee_id":'
    token2 = '}&f=["patent_number","patent_date","patent_processing_time","assignee_id","ipc_section","ipc_class","wipo_field_id"]&s=[{"patent_title":"asc"}]&o={"page":'
    token3 = ',"per_page":10000}'
    while x:
        url = token1 + el + token2 + str(page) + token3
        r = requests.get(url)
        html=r.text
        html = html.replace("null","None")
        count = ast.literal_eval(html)["count"]
        total_patent_count = ast.literal_eval(html)["total_patent_count"]
        if total_patent_count >100000:
            break
        if count<10000:
            x=False
        Dict = ast.literal_eval(html)["patents"]
        print(i,len(Dict))
        page+=1
        
        df_toadd = pd.read_json(json.dumps(Dict),orient='records')        
        df = pd.concat([df,df_toadd[(~df_toadd.patent_number.isin(L)) & (~df_toadd.patent_number.isin(added_elements))]])
        added_elements.extend(list((pd.read_json(json.dumps(Dict),orient='records')).patent_number.values))
        print("added_rows:",len(np.unique(added_elements)))
        
df.reset_index(drop=True)         

In [83]:
df

,patent_number,patent_date,patent_processing_time,ipc_section,ipc_class,assignee_id,IPCs,wipos,marked,assignees
0,5387603,1995-02-07,433,NaN,NaN,5330279e-a21e-438c-80d1-7e6ae793c092,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1.0,NaN
1,5771295,1998-06-23,552,NaN,NaN,2986e46e-7937-477f-9bc5-1f5eec05183f,"[{'ipc_section': 'H', 'ipc_class': '04'}]",[{'wipo_field_id': '2'}],1.0,NaN
2,5585587,1996-12-17,832,NaN,NaN,f7152515-d420-42d9-a7b6-3a2796bc5d10,"[{'ipc_section': 'G', 'ipc_class': '10'}]","[{'wipo_field_id': '34'}, {'wipo_field_id': '2...",1.0,NaN
3,5321490,1994-06-14,568,NaN,NaN,ec72750e-f789-4749-91f7-6292f7c6fda7,"[{'ipc_section': 'G', 'ipc_class': '01'}]","[{'wipo_field_id': '10'}, {'wipo_field_id': '1...",1.0,NaN
4,5750497,1998-05-12,1161,NaN,NaN,9a3afa40-66e5-4921-b831-64287f2052e0,"[{'ipc_section': 'C', 'ipc_class': '07'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
9995,7043712,2006-05-09,973,NaN,NaN,NaN,"[{'ipc_section': 'G', 'ipc_class': '06'}]","[{'wipo_field_id': '6'}, {'wipo_field_id': '9'}]",NaN,[{'assignee_id': 'ee0168d9-bf74-4513-bfbf-e73c...
9996,7334218,2008-02-19,1635,NaN,NaN,NaN,"[{'ipc_section': 'G', 'ipc_class': '06'}]",[{'wipo_field_id': '6'}],NaN,[{'assignee_id': 'ee0168d9-bf74-4513-bfbf-e73c...
9997,5355488,1994-10-11,294,NaN,NaN,NaN,"[{'ipc_section': 'G', 'ipc_class': '06'}]",[{'wipo_field_id': '6'}],NaN,[{'assignee_id': 'ee0168d9-bf74-4513-bfbf-e73c...
9998,9146828,2015-09-29,1119,NaN,NaN,NaN,"[{'ipc_section': 'G', 'ipc_class': '06'}, {'ip...","[{'wipo_field_id': '6'}, {'wipo_field_id': '12'}]",NaN,[{'assignee_id': 'ee0168d9-bf74-4513-bfbf-e73c...


# Nettoyage du DataFrame

In [49]:
ipc_section=[]
ipc_class=[]
for index,j in enumerate(df.IPCs.values):
    ipc = j
    for ele in ipc:
    print(ipc)

[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'C', 'ipc_class': '07'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'G', 'ipc_class': '10'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'C', 'ipc_class': '07'}, {'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'B', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'B', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'C', 'ipc_class': '12'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '09'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '08'}]
[{'ipc_section': 'G', 'ipc_class': '08'}]
[{'ipc_section': 'G', 'ipc_class': '08'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '0

[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '07'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'B', 'ipc_class': '67'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'C', 'ipc_class': '12'}]
[{'ipc_section': 'A', 'ipc_class': '23'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'B', 'ipc_class': '65'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '02'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'B', 'ipc_class':

[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '23'}]
[{'ipc_section': 'H', 'ipc_class': '02'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'D', 'ipc_class': '06'}]
[{'ipc_section': 'B', 'ipc_class': '21'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '05'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '23'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'B', 'ipc_class': '23'}]
[{'ipc_section': 'G', 'ipc_class': '02'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'B', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '23'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'C', 'ipc_class':

[{'ipc_section': 'A', 'ipc_class': '63'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '02'}]
[{'ipc_section': 'A', 'ipc_class': '47'}]
[{'ipc_section': 'G', 'ipc_class': '02'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'F', 'ipc_class': '16'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '01'}, {'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': 

[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'B', 'ipc_class': '29'}, {'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '10'}]
[{'ipc_section': 'F', 'ipc_class': '16'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'F', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'C', 'ipc_class': '08'}]
[{'ipc_section': 'G', 'ipc_class': '08'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '07'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '01'}, {'ipc_section': 'G', 'ipc_class': '02'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'H', 'ipc_class': '05'}]
[{'ipc_section': 'B', 'ipc_class': '05'}, {'ipc_section': 'A', 'ipc_class': '62'}]
[{'ipc_section': 'F', 'ipc_class': '16'

...

[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'B', 'ipc_class': '27'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'B', 'ipc_class': '60'}]
[{'ipc_section': 'G', 'ipc_class': '06'}, {'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'G', 'ipc_class': '06'}, {'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '08'}, {'ipc_section': 'G', 'ipc_class': '01'}, {'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'A', 'ipc_class': '63'}, {'ipc_section': 'G', 'ipc_class': '07'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '06'}

[{'ipc_section': 'H', 'ipc_class': '04'}, {'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'B', 'ipc_class': '65'}]
[{'ipc_section': 'B', 'ipc_class': '23'}, {'ipc_section': 'F', 'ipc_class': '16'}]
[{'ipc_section': 'G', 'ipc_class': '09'}, {'ipc_section': 'B', 'ipc_class': '23'}]
[{'ipc_section': 'G', 'ipc_class': '09'}, {'ipc_section': 'B', 'ipc_class': '23'}]
[{'ipc_section': 'A', 'ipc_class': '47'}]
[{'ipc_section': 'B', 'ipc_class': '62'}, {'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'A', 'ipc_class': '47'}]
[{'ipc_section': 'G', 'ipc_class': '06'}, {'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'G', 'ipc_class': '06'}, {'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'B', 'ipc_class': '66'}]
[{'ipc_section': 'F', 'ipc_class': '16'}, {'ipc_section': 'F', 'ipc_class': '28'}, {

[{'ipc_section': 'A', 'ipc_class': '47'}]
[{'ipc_section': 'H', 'ipc_class': '04'}, {'ipc_section': 'G', 'ipc_class': '02'}, {'ipc_section': 'G', 'ipc_class': '03'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'G', 'ipc_class': '10'}, {'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'H', 'ipc_class': '04'}, {'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'E', 'ipc_class': '04'}, {'ipc_section': 'B', 'ipc_class': '64'}, {'ipc_section': 'B', 'ipc_class': '32'}]
[{'ipc_section': 'G', 'ipc_class': '09'}, {'ipc_section': 'H', 'ipc_class': '04'}, {'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'H', 'ipc_class': '05'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'C', 'ipc_class': '11'}]
[{'ipc_section': 'A', 'ipc_class': '01'}, 

[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'A', 'ipc_class': '62'}]
[{'ipc_section': 'B', 'ipc_class': '65'}, {'ipc_section': 'B', 'ipc_class': '25'}, {'ipc_section': 'A', 'ipc_class': '47'}]
[{'ipc_section': 'G', 'ipc_class': '06'}, {'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'C', 'ipc_class': '12'}, {'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'F', 'ipc_class': '25'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'C', 'ipc_class': '08'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'F', 'ipc_class': '04'}]
[{'ipc_section': 'F', 'ipc_class': '04'}]
[{'ipc_section': 'F', 'ipc_class': '04'

[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'B', 'ipc_class': '25'}]
[{'ipc_section': 'G', 'ipc_class': '01'}, {'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'B', 'ipc_class': '26'}]
[{'ipc_section': 'B', 'ipc_class': '26'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'A', 'ipc_class': '21'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'A', 'ipc_class': '47'}]
[{'ipc_section': 'A', 'ipc_class': '47'}, {'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'B', 'ipc_class': '65'}]
[{'ipc_section': 'H', 'ipc_class': '2'}, {'ipc_section': 'H', 'ipc_class': '3'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]


In [47]:
df

,patent_number,patent_date,patent_processing_time,ipc_section,ipc_class,assignee_id,IPCs,wipos,marked
0,5387603,1995-02-07,433,NaN,NaN,5330279e-a21e-438c-80d1-7e6ae793c092,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1
1,5771295,1998-06-23,552,NaN,NaN,2986e46e-7937-477f-9bc5-1f5eec05183f,"[{'ipc_section': 'H', 'ipc_class': '04'}]",[{'wipo_field_id': '2'}],1
2,5585587,1996-12-17,832,NaN,NaN,f7152515-d420-42d9-a7b6-3a2796bc5d10,"[{'ipc_section': 'G', 'ipc_class': '10'}]","[{'wipo_field_id': '34'}, {'wipo_field_id': '2...",1
3,5321490,1994-06-14,568,NaN,NaN,ec72750e-f789-4749-91f7-6292f7c6fda7,"[{'ipc_section': 'G', 'ipc_class': '01'}]","[{'wipo_field_id': '10'}, {'wipo_field_id': '1...",1
4,5750497,1998-05-12,1161,NaN,NaN,9a3afa40-66e5-4921-b831-64287f2052e0,"[{'ipc_section': 'C', 'ipc_class': '07'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1
...,...,...,...,...,...,...,...,...,...
23905,10299691,2019-05-28,225,NaN,NaN,1b2d67d2-f1f9-4c29-8cd1-a5e75ba27be7,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...",[{'wipo_field_id': '13'}],1
23906,10292890,2019-05-21,966,NaN,NaN,f0997ba8-1ec2-4c0c-aaea-d80066a49317,"[{'ipc_section': 'A', 'ipc_class': '61'}]",[{'wipo_field_id': '13'}],1
23907,10299968,2019-05-28,343,NaN,NaN,e5fc32c3-a913-455a-94ce-9932d30eefff,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...","[{'wipo_field_id': '13'}, {'wipo_field_id': '6'}]",1
23908,10149616,2018-12-11,2133,NaN,NaN,4eb39673-7f59-4c45-bf6c-2275dccf9048,"[{'ipc_section': 'A', 'ipc_class': '61'}]",[{'wipo_field_id': '13'}],1


In [25]:
k=0
while len(df) < 50000:
    token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
    token2 = '}&f=["patent_number","patent_title","patent_date","patent_kind","patent_type","assignee_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
    url = token1 + str(L1[k:k+700]) + token2
    r = requests.get(url)
    html= r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["patents"]
    df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
    k+=700
    print(k)

700
1400
2100
2800
3500
4200
4900
5600
6300
7000
7700
8400
9100
9800
10500
11200
11900
12600
13300
14000
14700
15400
16100
16800
17500
18200
18900
19600
20300
21000
21700
22400
23100
23800
24500
25200
25900
26600


In [26]:
df

,patent_number,patent_title,patent_date,patent_kind,patent_type,assignees,marked
0,5387603,"1,5,7-trisubstituted indoline compounds and sa...",1995-02-07,A,utility,[{'assignee_id': '5330279e-a21e-438c-80d1-7e6a...,1.0
1,5771295,5-2-5 matrix system,1998-06-23,A,utility,[{'assignee_id': '2986e46e-7937-477f-9bc5-1f5e...,1.0
2,5585587,Acoustic image localization apparatus for dist...,1996-12-17,A,utility,[{'assignee_id': 'f7152515-d420-42d9-a7b6-3a27...,1.0
3,5321490,Active near-field object sensor and method emp...,1994-06-14,A,utility,[{'assignee_id': 'ec72750e-f789-4749-91f7-6292...,1.0
4,5750497,Acylated insulin,1998-05-12,A,utility,[{'assignee_id': '9a3afa40-66e5-4921-b831-6428...,1.0
...,...,...,...,...,...,...,...
694,5215272,Winding device having a tilting table and method,1993-06-01,A,utility,"[{'assignee_id': None, 'assignee_key_id': None}]",NaN
695,4034472,Wire termination apparatus,1977-07-12,A,utility,[{'assignee_id': '62769650-fc22-40db-a248-fb03...,NaN
696,10063472,Wireless communication terminal,2018-08-28,B2,utility,[{'assignee_id': '5718551a-fe57-489d-a94a-9286...,NaN
697,9933072,Working cylinder,2018-04-03,B2,utility,[{'assignee_id': 'fdb5f4af-1c7e-49cb-9716-cec9...,NaN


In [29]:
df = df.reset_index(drop=True)

In [30]:
df['marked'] = df['marked'].fillna(0)

In [36]:
max(df[df.marked==1].patent_date)

'2020-06-09'

In [32]:
df

,patent_number,patent_title,patent_date,patent_kind,marked,assignee_id
0,5387603,"1,5,7-trisubstituted indoline compounds and sa...",1995-02-07,A,1.0,5330279e-a21e-438c-80d1-7e6ae793c092
1,5771295,5-2-5 matrix system,1998-06-23,A,1.0,2986e46e-7937-477f-9bc5-1f5eec05183f
2,5585587,Acoustic image localization apparatus for dist...,1996-12-17,A,1.0,f7152515-d420-42d9-a7b6-3a2796bc5d10
3,5321490,Active near-field object sensor and method emp...,1994-06-14,A,1.0,ec72750e-f789-4749-91f7-6292f7c6fda7
4,5750497,Acylated insulin,1998-05-12,A,1.0,9a3afa40-66e5-4921-b831-64287f2052e0
...,...,...,...,...,...,...
50364,5215272,Winding device having a tilting table and method,1993-06-01,A,0.0,None
50365,4034472,Wire termination apparatus,1977-07-12,A,0.0,62769650-fc22-40db-a248-fb03540f65b3
50366,10063472,Wireless communication terminal,2018-08-28,B2,0.0,5718551a-fe57-489d-a94a-9286974683c0
50367,9933072,Working cylinder,2018-04-03,B2,0.0,fdb5f4af-1c7e-49cb-9716-cec98702c0c6


# Assignees df

In [75]:
df_assignees=pd.DataFrame(columns=["assignee_id", "assignee_organization","assignee_type", "assignee_lastknown_country", "assignee_total_num_patents", "assignee_total_num_inventors"])

In [76]:
#L_tmp = list(np.unique(df.assignee_id.dropna().values))
L2 = []
k=0
while k < len(assignees)-100:
    L2.append(json.dumps(assignees[k:k+100]))
    k+=100
L2.append(json.dumps(assignees[k:]))

In [77]:
for i in range(len(L2)):    
    token1 ='https://api.patentsview.org/assignees/query?q={"assignee_id":'
    token2 = '}&f=["assignee_id","assignee_organization","assignee_type","assignee_lastknown_country", "assignee_total_num_patents", "assignee_total_num_inventors"]&s=[{"assignee_organization":"asc"}]&o={"page":1,"per_page":1000}'
    url = token1 + L2[i] + token2
    r = requests.get(url)
    html=r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["assignees"]
    df_assignees = pd.concat([df_assignees,pd.read_json(json.dumps(Dict),orient='records')])
    print(i*100)
df_assignees=df_assignees.reset_index(drop=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [80]:
np.argmax(df_assignees.assignee_total_num_patents.values)

153

In [ ]:
df.to_csv('./info_patents.csv')
df_assignees.to_csv('./info_companies.csv')

In [82]:
df_assignees.iloc[153]

assignee_id                            ee0168d9-bf74-4513-bfbf-e73c933f90e7
assignee_organization           International Business Machines Corporation
assignee_type                                                             2
assignee_lastknown_country                                               US
assignee_total_num_patents                                           142160
assignee_total_num_inventors                                          63301
Name: 153, dtype: object